# Operações com Delta Lake

Este notebook demonstra as operações básicas de INSERT, UPDATE e DELETE usando Delta Lake com Apache Spark.

## Configuração do Ambiente

Primeiro, vamos configurar o ambiente e criar uma sessão Spark com suporte ao Delta Lake.

In [ ]:
# Importar as bibliotecas necessárias
import os
import sys
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit

# Adicionar o diretório src ao PYTHONPATH
project_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(os.path.join(project_dir, 'src'))

# Importar os módulos do projeto
from spark_delta_iceberg.spark_session import create_spark_session
from spark_delta_iceberg.delta_operations import DeltaLakeOperations
from spark_delta_iceberg.sample_data import create_sample_dataframe, create_sample_update_dataframe

# Criar a sessão Spark
spark = create_spark_session("DeltaLakeDemo")

# Criar uma instância da classe DeltaLakeOperations
delta_ops = DeltaLakeOperations(spark)

# Definir o caminho para a tabela Delta
delta_table_path = os.path.join(project_dir, "spark-warehouse", "vendas_delta")

print("Ambiente configurado com sucesso!")
print(f"Versão do Spark: {spark.version}")
print(f"Caminho da tabela Delta: {delta_table_path}")

## Criação de Dados de Exemplo

Vamos criar um DataFrame de exemplo com dados de vendas para usar em nossas operações.

In [ ]:
# Criar um DataFrame de exemplo
df_vendas = create_sample_dataframe(spark)

# Exibir o esquema
print("Esquema do DataFrame:")
df_vendas.printSchema()

# Exibir os dados
print("\nDados do DataFrame:")
df_vendas.show()

## Operação INSERT (Criação da Tabela Delta)

Vamos criar uma tabela Delta Lake com os dados de vendas.

In [ ]:
# Criar a tabela Delta com os dados de vendas
delta_ops.create_table(
    df=df_vendas,
    table_path=delta_table_path,
    mode="overwrite",
    partition_by=["estado"]
)

# Ler a tabela Delta para verificar
df_delta = delta_ops.read_table(delta_table_path)
print("\nDados da tabela Delta:")
df_delta.show()

## Operação INSERT (Adicionando Novos Dados)

Vamos adicionar novos dados à tabela Delta existente.

In [ ]:
# Criar novos dados para inserção
novos_dados = [
    (11, "Produto K", "Eletrônicos", 1800.00, "2023-01-24", "RS"),
    (12, "Produto L", "Alimentos", 45.60, "2023-01-25", "PR")
]

# Criar DataFrame com os novos dados
schema = "id INT, nome STRING, categoria STRING, preco DOUBLE, data_venda STRING, estado STRING"
df_novos = spark.createDataFrame(novos_dados, schema)

# Inserir os novos dados na tabela Delta
delta_ops.insert_data(
    table_path=delta_table_path,
    new_data=df_novos,
    mode="append"
)

# Ler a tabela Delta para verificar
df_delta_atualizado = delta_ops.read_table(delta_table_path)
print("\nDados da tabela Delta após inserção:")
df_delta_atualizado.show()

## Operação UPDATE

Vamos atualizar alguns dados na tabela Delta.

In [ ]:
# Atualizar o preço do Produto B
delta_ops.update_data(
    table_path=delta_table_path,
    condition="nome = 'Produto B'",
    update_expr={"preco": "99.90"}
)

# Atualizar o nome e o preço do Produto E
delta_ops.update_data(
    table_path=delta_table_path,
    condition="nome = 'Produto E'",
    update_expr={
        "nome": "'Produto E Premium'",
        "preco": "149.90"
    }
)

# Ler a tabela Delta para verificar
df_delta_atualizado = delta_ops.read_table(delta_table_path)
print("\nDados da tabela Delta após atualização:")
df_delta_atualizado.show()

## Operação DELETE

Vamos excluir alguns dados da tabela Delta.

In [ ]:
# Excluir produtos da categoria Alimentos com preço menor que 30
delta_ops.delete_data(
    table_path=delta_table_path,
    condition="categoria = 'Alimentos' AND preco < 30"
)

# Ler a tabela Delta para verificar
df_delta_atualizado = delta_ops.read_table(delta_table_path)
print("\nDados da tabela Delta após exclusão:")
df_delta_atualizado.show()

## Operação MERGE

Vamos realizar uma operação MERGE para atualizar e inserir dados em uma única operação.

In [ ]:
# Criar um DataFrame com dados para atualização e inserção
df_update = create_sample_update_dataframe(spark)
print("\nDados para MERGE:")
df_update.show()

# Realizar a operação MERGE
delta_ops.merge_data(
    table_path=delta_table_path,
    source_df=df_update,
    merge_condition="target.id = source.id",
    matched_update={
        "nome": "source.nome",
        "preco": "source.preco"
    },
    not_matched_insert=True
)

# Ler a tabela Delta para verificar
df_delta_atualizado = delta_ops.read_table(delta_table_path)
print("\nDados da tabela Delta após MERGE:")
df_delta_atualizado.show()

## Time Travel

Vamos explorar o recurso de Time Travel do Delta Lake para acessar versões anteriores dos dados.

In [ ]:
# Obter o histórico da tabela
history_df = delta_ops.get_history(delta_table_path)
print("\nHistórico da tabela Delta:")
history_df.select("version", "timestamp", "operation", "operationParameters").show(truncate=False)

# Acessar a versão 0 (inicial) da tabela
df_versao_0 = delta_ops.time_travel(delta_table_path, version=0)
print("\nDados da versão 0 da tabela Delta:")
df_versao_0.show()

# Acessar a versão 1 da tabela (após a primeira inserção)
df_versao_1 = delta_ops.time_travel(delta_table_path, version=1)
print("\nDados da versão 1 da tabela Delta:")
df_versao_1.show()

## Conclusão

Neste notebook, demonstramos as operações básicas de INSERT, UPDATE e DELETE usando Delta Lake com Apache Spark. Também exploramos recursos avançados como MERGE e Time Travel.

O Delta Lake oferece várias vantagens para o gerenciamento de dados em data lakes:

1. **Transações ACID**: Garantem a consistência dos dados mesmo em caso de falhas.
2. **Time Travel**: Permite acessar versões anteriores dos dados para auditoria ou rollback.
3. **Operações DML**: Suporte a operações como UPDATE, DELETE e MERGE, que não são nativas em formatos tradicionais de data lake.
4. **Evolução de Esquema**: Permite alterar o esquema dos dados sem afetar os consumidores.
5. **Unificação de Batch e Streaming**: Permite processar dados em batch e streaming de forma unificada.